In [5]:
#pip install torch
#pip install gpytorch

SyntaxError: invalid syntax (<ipython-input-5-b580e7aeeb51>, line 2)

In [1]:
from platform import python_version
print(python_version())

3.6.13


In [2]:
import os
import numpy as np
import torch
import gpytorch
from sklearn.metrics import mean_absolute_error, mean_squared_error
import pickle

float_type = torch.float



In [4]:
class GPDTemplate:
    def __init__(
        self,
        M,
        emb_sizes,
        batch_size=512,
        te_size=None,
        obs_mean=None,
        emb_reg=1e-3,
        lr=1e-3,
        ARD=True,
        svgp=True,
        save_path="./",
    ):
        """
        :param M: integer, number of inducing points.
        :param emb_sizes: a list of embedding sizes as integers.
        :param batch_size: integer, mini batch size for training, necessary to define the placeholders.
        :param te_size: integer, batch size for testing, necessary to define the placeholders.
        :param obs_mean: integer, mean of training targets, optional.
        :param emb_reg: float, regularization term for embeddings.
        :param lr: float, learning rate.
        :param ARD: boolean, ARD parameter in gpflow.kernel.
        :param svgp: boolean, if True then apply svgp, sgpr otherwise.
        :param save_path: string, path to save the trained models.
        """
        self.M = M
        self.emb_sizes = emb_sizes
        self.batch_size = batch_size
        self.te_size = te_size
        self.obs_mean = obs_mean
        self.emb_reg = emb_reg
        self.lr = lr
        self.ARD = ARD
        self.svgp = svgp
        self.save_path = save_path
        self.param_ids = list(locals().keys())
        

    def build(self):
        """
        To be initiated by children classes.
        :return: None
        
        tf.reset_default_graph()
        gpflow.reset_default_graph_and_session()
        """

    def save(self):
        """
        Save trained models.
        :return: None
        """
        params = {}
        for v in self.param_ids:
            if v not in ["self", "kwargs", "__class__"]:
                params[v] = self.__getattribute__(v)
        if not os.path.exists(self.save_path):
            os.makedirs(self.save_path)
        save_name = os.path.join(self.save_path, "class_params.pkl")
        with open(save_name, "wb") as handle:
            pickle.dump(params, handle)

    def load_params(self):
        """
        To be initiated by children classes.
        :return: None
        """
        return None
    

class GPD(GPDTemplate):
    def __init__(self, I, J, K, **kwargs):
        """
        :param I: integer, number of entities in the first dimension.
        :param J: integer, number of entities in the second dimension.
        :param K: integer, number of entities in the third dimension.
        :param kwargs: a dictionary for training hyper parameters, forwarded to GPDTemplate.__init__().
        """
        super(GPD, self).__init__(**kwargs)

        self.I = I
        self.J = J
        self.K = K
        self.param_ids.extend(list(locals().keys()))

        # Placeholders:
        self.index_1 = None
        self.index_2 = None
        self.index_3 = None
        self.y = None

        self.index_1_te = None
        self.index_2_te = None
        self.index_3_te = None

        self.index_1_M = None
        self.index_2_M = None
        self.index_3_M = None

        self.kernel_ids = None
        self.emb1 = None
        self.emb2 = None
        self.emb3 = None

        self.e1 = None
        self.e2 = None
        self.e3 = None
        self.e1_te = None
        self.e2_te = None
        self.e3_te = None
        self.e1_M = None
        self.e2_M = None
        self.e3_M = None

        self.h = None
        self.h_te = None
        self.h_M = None

        self.kernel = None
        self.gp_model = None
        self.loss = None

        self.ym = None
        self.yv = None
        self.ym_te = None
        self.yv_te = None

        self.opt_step = None
        self.sess = None

